In [1]:
def prepare_dataset(raw):
    sentences, relations = [], []
    to_replace = [("\"", ""), ("\n", ""), ("<", " <"), (">", "> ")]
    last_was_sentence = False
    for line in raw:
        sl = line.split("\t")
        if last_was_sentence:
            relations.append(sl[0].split("(")[0].replace("\n", ""))
            last_was_sentence = False
        if sl[0].isdigit():
            sent = sl[1]
            for rp in to_replace:
                sent = sent.replace(rp[0], rp[1])
            sentences.append(sent)
            last_was_sentence = True
    print("Found {} sentences".format(len(sentences)))
    return sentences, relations

In [2]:
n_most_common_words = 20000 # vocabulary size
max_len = 100
# Initialization
tokenizer = Tokenizer(num_words=n_most_common_words, filters='!"#$%&()*+,-./:;=?@[]^_`{|}~', lower=True)
# Fit and transformation
tokenizer.fit_on_texts(df['sentences'].values)
sequences = tokenizer.texts_to_sequences(df['sentences'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
# Padding
X = pad_sequences(sequences, maxlen=max_len)

NameError: name 'Tokenizer' is not defined

In [ ]:
labels = to_categorical(df['labels'], num_classes=len(df.labels.unique())) # from keras.utils.np_utils


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X , labels, test_size=0.20, random_state=42)


In [ ]:
embedding_size = 300
model = Sequential()
model.add(Embedding(n_most_common_words, embedding_size, input_length=X.shape[1]))
model.add(Bidirectional(LSTM(128, dropout=0.7, recurrent_dropout=0.7)))
model.add(Dense(labels.shape[1], activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
print(model.summary())

In [ ]:
model.fit(X_train, y_train, epochs=3, batch_size=128, validation_split=0.2)


In [ ]:
prediction_probas = model.predict(X_test) 
predictions = [np.argmax(pred) for pred in predicion_probas]

print(confusion_matrix(y_test, predictions)) #from sklearn.metrics
print(classification_report(y_test, predictions, digits=3))